<a href="https://colab.research.google.com/github/imabari/rakuten_tools/blob/main/saitama_mymap_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile

In [2]:
import requests
import pandas as pd
from lxml import etree

# 設定

In [3]:
# マイマップのmid
mid = "1VFhvD6b5JpMJb3Jr-gXM8FRm1XJjQzZe"
lid = "1dZ2-7u5FSY"

# マイマップからKMZをダウンロード

In [4]:
url = f"https://www.google.com/maps/d/kml?mid={mid}"

# レイヤ指定
# url = f"https://www.google.com/maps/d/kml?mid={mid}&lid={lid}"

In [5]:
!wget "$url" -O rakuten.kmz

--2021-12-20 12:40:25--  https://www.google.com/maps/d/kml?mid=1VFhvD6b5JpMJb3Jr-gXM8FRm1XJjQzZe
Resolving www.google.com (www.google.com)... 108.177.127.104, 108.177.127.106, 108.177.127.147, ...
Connecting to www.google.com (www.google.com)|108.177.127.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/vnd.google-earth.kmz]
Saving to: ‘rakuten.kmz’

rakuten.kmz             [ <=>                ] 255.42K  --.-KB/s    in 0.003s  

2021-12-20 12:40:27 (88.3 MB/s) - ‘rakuten.kmz’ saved [261551]



# KMZからCSV変換

In [6]:
with zipfile.ZipFile("rakuten.kmz") as existing_zip:
    existing_zip.extractall("kmz")

In [7]:
ns = {"kml": "http://www.opengis.net/kml/2.2"}
tree = etree.parse("./kmz/doc.kml")

In [8]:
data = []

for i in tree.xpath("//kml:Placemark", namespaces=ns):

    d = {}

    d["場所"] = "".join(i.xpath("./kml:name/text()", namespaces=ns)).strip()
    d["geometry"] = "".join(
        i.xpath("./kml:Point/kml:coordinates/text()", namespaces=ns)
    ).strip()

    for j in i.xpath("./kml:ExtendedData/kml:Data", namespaces=ns):

        k = j.attrib["name"].strip()
        v = "".join(j.xpath("./kml:value/text()", namespaces=ns)).strip()

        d[k] = v

    data.append(d)

In [9]:
df0 = pd.DataFrame(data)

In [10]:
df0.shape

(2032, 17)

In [11]:
df0.to_csv("all.csv")

In [12]:
df0[["経度", "緯度", "標高"]] = df0["geometry"].str.split(",", expand=True)

In [13]:
# 文字から数値に変換
df0["経度"] = pd.to_numeric(df0["経度"])
df0["経度"] = pd.to_numeric(df0["経度"])

In [14]:
# 緯度・経度が欠損の場合は削除
df0.dropna(subset=["X", "Y"], how="any", inplace=True)
df0

,場所,geometry,Type (自動生成),X,Y,eNBID,CID,ANT種別,Status,都道府県,市町村,字町名,提供者その他,発見日,開局日,区分,備考,経度,緯度,標高
184,(206) 双柳なみやなぎ,"139.335056,35.848168,0",4G撤去,139.335056,35.848168,,,4G,撤去,埼玉県1,飯能市,双柳なみやなぎ,@yuzurinco155 撤去確認2021/11/28,2021/05/26,,,NaN,139.335056,35.848168,0
185,(25) 田島9丁目,"139.6253541,35.84671739,0",4G撤去,139.6253541,35.84671739,,,4G,撤去,埼玉県1,さいたま市桜区,田島9丁目,"@lolsamurai 旧344143-4,5,6",2021/11/23,2020/07/10,,NaN,139.625354,35.84671739,0
186,"262280-10,11,12 栄-2","139.6790657,36.168588,0",4G開局,139.6790657,36.168588,262280.0,"10,11,12",4G,開局,埼玉県1,加須市,栄-2,@chachabokuji @InappyRakuten,2021/04/29,2021/07/07,,NaN,139.679066,36.168588,0
187,"262280-7,8,9 飯積","139.632908,36.19303,0",4G開局,139.632908,36.19303,262280.0,"7,8,9",4G,開局,埼玉県1,加須市,飯積,@InappyRakuten,2021/07/07,2021/07/07,,NaN,139.632908,36.19303,0
188,"262281-4,5,6 栄-1","139.660362,36.1780414,0",4G開局,139.660362,36.1780414,262281.0,"4,5,6",4G,開局,埼玉県1,加須市,栄-1,@chachabokuji @InappyRakuten,2021/04/29,2021/07/07,,NaN,139.660362,36.1780414,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2027,(91) 鬼石,"139.058287,36.159373,0",4G未開局,139.058287,36.159373,,,4G,未開局,群馬県,藤岡市,鬼石,NaN,2021/9/19,,,@InappyRakuten,139.058287,36.159373,0
2028,"262314-10,11,12 南町","139.719635,36.189693,0",4G開局,139.719635,36.189693,262314,101112.0,4G,開局,茨城県,古河市,南町,NaN,2021/9/25,2021/9/25,,@InappyRakuten,139.719635,36.189693,0
2029,"262318-7,8,9 中央町2丁目","139.700563,36.193835,0",4G開局,139.700563,36.193835,262318,789.0,4G,開局,茨城県,古河市,中央町2丁目,NaN,2021/9/25,2021/9/25,,@InappyRakuten,139.700563,36.193835,0
2030,"262325-1,2,3 原町","139.70984,36.180736,0",4G開局,139.70984,36.180736,262325,123.0,4G,開局,茨城県,古河市,原町,NaN,2021/7/7,2021/10/14,,@InappyRakuten,139.709840,36.180736,0


In [15]:
# 抽出条件
df1 = df0[df0["ANT種別"] == "4G"].copy().reset_index(drop=True)

In [16]:
df1

,場所,geometry,Type (自動生成),X,Y,eNBID,CID,ANT種別,Status,都道府県,市町村,字町名,提供者その他,発見日,開局日,区分,備考,経度,緯度,標高
0,(206) 双柳なみやなぎ,"139.335056,35.848168,0",4G撤去,139.335056,35.848168,,,4G,撤去,埼玉県1,飯能市,双柳なみやなぎ,@yuzurinco155 撤去確認2021/11/28,2021/05/26,,,NaN,139.335056,35.848168,0
1,(25) 田島9丁目,"139.6253541,35.84671739,0",4G撤去,139.6253541,35.84671739,,,4G,撤去,埼玉県1,さいたま市桜区,田島9丁目,"@lolsamurai 旧344143-4,5,6",2021/11/23,2020/07/10,,NaN,139.625354,35.84671739,0
2,"262280-10,11,12 栄-2","139.6790657,36.168588,0",4G開局,139.6790657,36.168588,262280.0,"10,11,12",4G,開局,埼玉県1,加須市,栄-2,@chachabokuji @InappyRakuten,2021/04/29,2021/07/07,,NaN,139.679066,36.168588,0
3,"262280-7,8,9 飯積","139.632908,36.19303,0",4G開局,139.632908,36.19303,262280.0,"7,8,9",4G,開局,埼玉県1,加須市,飯積,@InappyRakuten,2021/07/07,2021/07/07,,NaN,139.632908,36.19303,0
4,"262281-4,5,6 栄-1","139.660362,36.1780414,0",4G開局,139.660362,36.1780414,262281.0,"4,5,6",4G,開局,埼玉県1,加須市,栄-1,@chachabokuji @InappyRakuten,2021/04/29,2021/07/07,,NaN,139.660362,36.1780414,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1627,(91) 鬼石,"139.058287,36.159373,0",4G未開局,139.058287,36.159373,,,4G,未開局,群馬県,藤岡市,鬼石,NaN,2021/9/19,,,@InappyRakuten,139.058287,36.159373,0
1628,"262314-10,11,12 南町","139.719635,36.189693,0",4G開局,139.719635,36.189693,262314,101112.0,4G,開局,茨城県,古河市,南町,NaN,2021/9/25,2021/9/25,,@InappyRakuten,139.719635,36.189693,0
1629,"262318-7,8,9 中央町2丁目","139.700563,36.193835,0",4G開局,139.700563,36.193835,262318,789.0,4G,開局,茨城県,古河市,中央町2丁目,NaN,2021/9/25,2021/9/25,,@InappyRakuten,139.700563,36.193835,0
1630,"262325-1,2,3 原町","139.70984,36.180736,0",4G開局,139.70984,36.180736,262325,123.0,4G,開局,茨城県,古河市,原町,NaN,2021/7/7,2021/10/14,,@InappyRakuten,139.709840,36.180736,0


# 距離

In [18]:
!pip install pyproj

     |████████████████████████████████| 6.3 MB 13.1 MB/s 


In [19]:
import pyproj
import itertools

In [20]:
from tqdm.notebook import tqdm

In [21]:
grs80 = pyproj.Geod(ellps="GRS80")

In [22]:
data1 = []

for i in tqdm(itertools.permutations(df1.index, 2)):

    a = df1.loc[i[0]]
    b = df1.loc[i[1]]
    
    data1.append([a["場所"], b["場所"], grs80.inv(a.X, a.Y, b.X, b.Y)[2]])

0it [00:00, ?it/s]

In [23]:
df2 = pd.DataFrame(data1, columns=["A", "B", "distance"])

In [24]:
dg2 = df2.groupby(by="A")

In [25]:
df2_min = df2.loc[dg2["distance"].idxmin(), :].sort_values("distance")
df2_min.head(20)

,A,B,distance
2542656,手子生,(229) 手子生,2.412218
2544287,(229) 手子生,手子生,2.412218
1409076,"344334-7,8,9 粕壁3丁目-1",(63) 粕壁3丁目-2,112.281685
2486507,(63) 粕壁3丁目-2,"344334-7,8,9 粕壁3丁目-1",112.281685
2622623,"1966616-10,11,12 中央１丁目",(197) 住吉町,127.038300
2620992,(197) 住吉町,"1966616-10,11,12 中央１丁目",127.038300
199102,"279838-4,5 宮町1丁目",279838-1 大門町2丁目,170.781073
195841,279838-1 大門町2丁目,"279838-4,5 宮町1丁目",170.781073
2523892,(24) 美杉台,"344295-1,2,3 美杉台1丁目",176.539745
1200331,"344295-1,2,3 美杉台1丁目",(24) 美杉台,176.539745


In [26]:
df2_min.tail(20)

,A,B,distance
2645471,(113) 四方津,(113) 上野原,3504.744912
2539394,"262149-1,2,3 高須賀",(229) 手子生,4353.802195
2619361,(197) 下五貫野,"1966616-10,11,12 中央１丁目",4503.833290
2650369,(113) 笹子町,(113) 大和町初鹿野,5041.257154
2605893,"344297-1,2,3 東上之宮町","344430-1,2,3 金久保",5731.136194
2648736,(113) 初狩町中初狩,(113) 笹子町,5757.474373
2630773,"245808-4,5,6 迫間町","229427-7,8,9 足次町",5929.647397
2604670,馬場524,"262235-1,2,3 弓田",6315.550452
2655035,(91) 鬼石,"344492-7,8,9 本野上",6969.881238
2647103,(113) 富浜町鳥沢,(113) 四方津,7326.719980


In [27]:
df2_min[(df2_min["A"].str.contains("御堂")) & (df2_min["B"].str.contains("勝呂"))]

,A,B,distance
2323817,"344502-1,2,3 御堂","344458-10,11,12 勝呂",3318.963425
